<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Semana07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
# importações
import matplotlib.pyplot as plt
import random 
import numpy as np
import time 
import copy
import imageio
import os
from google.colab import files

def cria_gif(arquivos, nome_gif):
  imagens = []
  for arquivo_png in arquivos:
      imagens.append(imageio.imread(arquivo_png))
  duration = 1
  imageio.mimwrite(nome_gif, imagens, duration=duration, loop=3)
  for arquivo_png in arquivos:
      os.remove(arquivo_png)
        
def grafico(populacao, TP, g, arquivos):
  fig, ax = plt.subplots(figsize=(10,4))       # cria objeto figura e os eixos da figura
  
  # funções de uma variável y = f(x)
  x = np.arange(0, 31.0, 0.1) 
  y = funcao(x)
  ax.plot(x, y)  # estabelece a ordem dos eixos e faz o gráfico
  
  for i in range(TP):
      xi = populacao[i][1]
      yi = populacao[i][2]
      ax.plot([xi],[yi], "ro")       
  ax.grid(True, linestyle='-')  # formato da grade de fundo (opcional)
  nome = "GA"+str(g)+".png"
  arquivos.append(nome)
  fig.savefig(nome) #, dpi=fig.dpi)
  plt.close(fig)
  
def funcao(x):
  return np.sin(np.pi*x) - (1/20)*x**2 + x

def criaIndividuo(N): # cria uma cadeia binária aleatória
  individuo = [random.randint(0,1) for b in range(N)]
  return individuo

def criaPopulacao(TP, N):
  # [[lista binária], inteiro decodificado, função objetivo]
  # 1 indivíduo: [[1,1,1,1,1], 31, -17.049999999999997]
  populacao = [[criaIndividuo(N), 0, 0] for i in range(TP)]
  return populacao

def decodificaIndividuo(individuo, N):
  inteiro = 0

  for b in range(N):
    bit = individuo[b]
    potencia = N - 1 - b
    inteiro = inteiro + bit*(2**potencia)

  return inteiro

def avaliaPopulacao(populacao, N, TP):
  for i in range(TP):
    inteiro = decodificaIndividuo(populacao[i][0], N) # só a cadeia
    fitness = funcao(inteiro)
    populacao[i][1] = inteiro
    populacao[i][2] = fitness
        
def torneioBinario(populacao, TP):
  paisSelecionados = []

  todos = list(range(TP))
  for i in range(TP):
    indAleat1 = random.choice(todos)
    indAleat2 = random.choice(todos)
    while (indAleat1 == indAleat2):
      indAleat2 = random.choice(todos)

    if (populacao[indAleat1][2] > populacao[indAleat2][2]):
      paiSelecionado = copy.deepcopy(populacao[indAleat1])
    else:
      paiSelecionado = copy.deepcopy(populacao[indAleat2])

    paisSelecionados.append(paiSelecionado)
  return paisSelecionados

def crossover(paisSelecionados, TP, N):
  # paisSelecionados = [[[0,1,0,1,0], x, fitness]],[],[]]
  PC = 0.95
  filhos = []

  for i in range(TP//2): # 2 pais geram 2 filhos
    doisPais = random.sample(paisSelecionados, 2) 
    aleatorio = random.random() # entre 0 e 1
    if aleatorio <= PC: # crossover acontece
      ponto = random.randint(1, N-1)
      filho1 = doisPais[0][0][:ponto] + doisPais[1][0][ponto:]
      filho2 = doisPais[1][0][:ponto] + doisPais[0][0][ponto:]
    else:
      filho1 = doisPais[0][0]
      filho2 = doisPais[1][0]
    filhos.append([filho1, 0, 0])
    filhos.append([filho2, 0, 0])

  return filhos

def mutacao(filhos, TP, N):
  PM = 0.1
  for i in range(TP):
    for b in range(N):
      aleatorio = random.random()
      if aleatorio <= PM:
        filhos[i][0][b] = 1 - filhos[i][0][b] # transforma 0 em 1 e 1 em 0. pense!!!




def main():
  # constantes
  N = 5          # quantidade de bits de um indivíduo
  TP = 10        # tamanho da população do GA
  G = 20         # número de gerações do GA
  random.seed()  # inicia a semente dos número pseudo randômicos
  arquivos = []
  nome = "filme.gif"
  
  inicio = time.time()  # começa a contar o tempo
  
  # Inicializa População 
  populacao = criaPopulacao(TP, N)
  

  # Avalia População
  avaliaPopulacao(populacao, N, TP)
  grafico(populacao, TP, 0, arquivos)
  
  for g in range(1, G):  # gerações
      # seleciona pares de pais aleatoriamente
      paisSelecionados = torneioBinario(populacao, TP)
      
      # crossover
      filhos = crossover(paisSelecionados, TP, N)
      
      # mutação
      mutacao(filhos, TP, N)

      # com filhos: Avalia
      avaliaPopulacao(filhos, N, TP)

      # populacao próxima geração, seleção: pais e filhos
      mista = populacao + filhos
      mistaOrd = sorted(mista, key=lambda tup: tup[2], reverse=True)
      populacao = mistaOrd[:TP]
      grafico(populacao, TP, g, arquivos)

  # fim do GA
  fim = time.time()
  print("Tempo de execução (s): ", fim - inicio)
  cria_gif(arquivos, nome)
  files.download('filme.gif')

In [ ]:
main()

Tempo de execução (s):  7.341706991195679


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>